In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

23/08/15 18:04:09 WARN Utils: Your hostname, Blade-15-Base-Model resolves to a loopback address: 127.0.1.1; using 192.168.1.231 instead (on interface wlo1)
23/08/15 18:04:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/15 18:04:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

23/08/15 18:04:13 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

# IMPORTANT 

*   You need to train and test a machine learning model in SPARK to have 2 columns only : label and features (ACTUAL FORMAT SPARK NEEDS)
*   numerical label
*   feature : vector, we gonna use features transformation


In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction') # 3 paramètres

In [7]:
# next step : train the model with the data

lrModel = lr.fit(training)

23/08/15 18:04:18 WARN Instrumentation: [b0b526db] regParam is zero, which might cause numerical instability and overfitting.


In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

# SPLIT TRAIN TEST

In [13]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

23/08/15 18:04:19 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [14]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [15]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
+----------

In [16]:
correct_model = lr.fit(train_data)

23/08/15 18:04:20 WARN Instrumentation: [f5cf5322] regParam is zero, which might cause numerical instability and overfitting.


In [17]:
test_results = correct_model.evaluate(test_data)

In [18]:
test_results.rootMeanSquaredError

10.572125351769705

In [19]:
unlabeled_data = test_data.select('features')

In [20]:
predictions = correct_model.transform(test_data)

In [21]:
predictions.show()

+-------------------+--------------------+--------------------+
|              label|            features|          prediction|
+-------------------+--------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...| -0.4959553538834093|
| -23.51088409032297|(10,[0,1,2,3,4,5,...| -1.9871737355839834|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|  0.8795251674255697|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|-0.07455394339135424|
|-19.782762789614537|(10,[0,1,2,3,4,5,...| 0.22292132452708713|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|  0.5293341217409917|
|-17.026492264209548|(10,[0,1,2,3,4,5,...| -1.0116987551321324|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|   3.242339418768169|
|-15.359544879832677|(10,[0,1,2,3,4,5,...|  1.7330801164848038|
|-15.310980589416289|(10,[0,1,2,3,4,5,...| -2.3407154713370444|
|-13.976130931152703|(10,[0,1,2,3,4,5,...|  0.6729143018533165|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|  3.8920282517758156|
|-13.039928064104615|(10,[0,1,2,3,4,5,..